# Obstacle avoidance block

## Imports

Import the script `thymio.py` and other libraries 

In [1]:
import os
import sys
import time
import serial
import math

# Variable to know if Thymio has already been connected before or not (== 1 if it is the case)
try:
    CONNECC
except NameError:
    CONNECC = 0
    print('Thymio will be connected.')

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
#sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

# Print the path to Python3 executable
# print(sys.executable)

Thymio will be connected.


## Thymio connection

First connect Thymio to the computer on one USB port through the USB cable.

### Look-up table for ports:
Replace `/dev/cu.usbmodem142101` (or current port value) below by the correct computer port:

Lucas: `\\.\COM3` for USB-cable and `\\.\COM4` for USB-dongle

Emma: ` `

Elise: ` `

Océane: ` `

In [2]:
if CONNECC == 0:
    CONNECC = 1
    th = Thymio.serial(port="\\.\COM4", refreshing_rate=0.1)
    time.sleep(1)

In [3]:
print(CONNECC)

1


## Tests

Testing communication with Thymio.

In [4]:
# Defines if tests should be run
TEST = 1

In [16]:
th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)

### Retrieve values

In [6]:
if TEST == 1:
    for i in range(10):
        print(th["prox.horizontal"])
        time.sleep(0.2)

[0, 0, 4423, 0, 0, 0, 0]
[0, 0, 4422, 0, 0, 0, 0]
[0, 0, 4424, 0, 0, 0, 0]
[0, 0, 4425, 0, 0, 0, 0]
[0, 0, 4425, 0, 0, 0, 0]
[0, 0, 4425, 0, 0, 0, 0]
[0, 0, 4421, 0, 0, 0, 0]
[0, 0, 4423, 0, 0, 0, 0]
[0, 0, 4421, 0, 0, 0, 0]
[0, 0, 4423, 0, 0, 0, 0]


### Set values

In [7]:
if TEST == 1:
    th.set_var("motor.left.target", 100)
    th.set_var("motor.right.target", 100)

    time.sleep(1)

    th.set_var("motor.left.target", 0)
    th.set_var("motor.right.target", 0)

## Obstacle avoidance algorithm

The robot can be in two states: global navigation or obstacle avoidance when one is detected by the sensors (caution for not confounding walls from the map and unknown obstacles).

When in obstacle avoidance state, the robot accelerates the wheels close to the obstacles.

In [8]:
# Tests

#dir(th)
#th.variable_description()
# Conio
# th.set_var("mic.intensity", 0)

th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)

In [15]:
State = 1 # State == 0 => Thymio goes straight, State == 1 => Thymio is in obstacle avoidance mode
Velocity_0 = 100
Obstacle_gain = [10, 15]
Threshold_enter_obstavoid = 20 # Threshold to enter obstacle avoidance mode
Threshold_quit_obstavoid = 10 # Threshold to quit obstacle avoidance mode
Timer = 1 * 10**(-3) # 1ms
Counter = 0
Limit_loops = 10000 # number of while loops to go through before quitting, entering obstacle avoidance mode reset the counter

while Counter < Limit_loops:
    if (State == 0 and
        (th["prox.horizontal"][0] > Threshold_enter_obstavoid or
        th["prox.horizontal"][1] > Threshold_enter_obstavoid or
        th["prox.horizontal"][2] > Threshold_enter_obstavoid or
        th["prox.horizontal"][3] > Threshold_enter_obstavoid or
        th["prox.horizontal"][4] > Threshold_enter_obstavoid)): # enter obstacle avoidance
        State = 1
    elif (State == 1 and
        th["prox.horizontal"][0] < Threshold_quit_obstavoid and
        th["prox.horizontal"][1] < Threshold_enter_obstavoid and
        th["prox.horizontal"][2] < Threshold_enter_obstavoid and
        th["prox.horizontal"][3] < Threshold_enter_obstavoid and
        th["prox.horizontal"][4] < Threshold_quit_obstavoid): # quit obstacle avoidance
        State = 0
    
    if State == 0: # goes straight
        th.set_var("motor.right.target", Velocity_0)
        th.set_var("motor.left.target", Velocity_0)
        Counter += 1
    else: # avoid obstacle depending on sensors values
        if (th["prox.horizontal"][0] == 0 and
            th["prox.horizontal"][1] == 0 and
            th["prox.horizontal"][3] == 0 and
            th["prox.horizontal"][4] == 0):
            Left_gain = 0
            Right_gain = int(Obstacle_gain[1]*(th["prox.horizontal"][2]/100))
        else:
            Left_gain = int(Obstacle_gain[0]*(th["prox.horizontal"][4]/100)) + int(Obstacle_gain[1]*(th["prox.horizontal"][3]/100))
            Right_gain = int(Obstacle_gain[0]*(th["prox.horizontal"][0]/100)) + int(Obstacle_gain[1]*(th["prox.horizontal"][1]/100))
        
        if Left_gain > Velocity_0:
            th.set_var("motor.left.target", 2**16 + (Velocity_0 - Left_gain))
        else:
            th.set_var("motor.left.target", (Velocity_0 - Left_gain))
        
        if Right_gain > Velocity_0:
            th.set_var("motor.right.target", 2**16 + (Velocity_0 - Right_gain))
        else:
            th.set_var("motor.right.target", (Velocity_0 - Right_gain))
        Counter = 0
        
    time.sleep(Timer) # wait 1ms before relooping


th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)
    

KeyboardInterrupt: 

## Calibration Odometry

~ 0.0325 cm/s for 1 motor input unit of Thymio

rotation: ~9.32s for 360° with [100, -100] motor inputs => ~ 0.0067 rad/s for [1, -1] motor inputs

In [10]:
# speed = []
# omega = 2*3.1415926/9.32
# omega_thymio = omega/100
# print(omega_thymio)

In [11]:
counter = 0
VelLeft = 100
VelRight = -100
adjust = 1.9 #4.15
SpeedGain = adjust*0.0325 * 10**(-3)
timer = 1 * 10**(-3) # 1ms
b = 0.095 # distance between Thymio's wheels in m

X = 0
Y = 0
Theta = 0

t = 0

while counter < 1000:
    #a = time.time()
    counter += 1
    th.set_var("motor.right.target", VelRight)
    th.set_var("motor.left.target", VelLeft)

    DTheta = (VelLeft*SpeedGain - VelRight*SpeedGain)/(2*b)
    DS = (VelLeft + VelRight)/2
    
    Theta += DTheta
    X += DS*SpeedGain*math.cos(math.radians(Theta))
    Y += DS*SpeedGain*math.sin(math.radians(Theta))

    time.sleep(timer)
    #b = time.time()
    #t += b-a

#print(t)
th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)

print(X)
print(Y)
print(Theta)

ValueError: bytes must be in range(0, 256)

In [ ]:
# t = []
# for i in range(1,1000):
#     a = time.time()
#     time.sleep(0.001)
#     b = time.time()
#     t.append(b-a)
# 
# print(sum(t)/1000)

In [ ]:
# delta_threshold = 300
# square = 5 # cm
# 
# count = 0
# current_square = 1
# previous_square = current_square
# change = []
# vel = 100
# start_time =  time.time()
# #print(th["prox.ground.delta"])
# 
# while (count < 3):
#     th.set_var("motor.left.target", vel)
#     th.set_var("motor.right.target", vel)
#     ground = th["prox.ground.delta"]
#     if (time.time() - start_time) > 30:
#         print('TIMED OUT')
#         break
#     if (ground[0] < delta_threshold and ground[1] < delta_threshold):
#         current_square = 0
#     else:
#         current_square = 1
#     if current_square != previous_square:
#         start_time = time.time()
#         change.append(time.time())
#         previous_square = current_square
#         count += 1
# 
# th.set_var("motor.left.target", 0)
# th.set_var("motor.right.target", 0)
# 
# first = change[0]
# change = [x - first for x in change]
# for i in range(0, len(change) - 1):
#         speed.append(square/(change[i+1]-change[i])/vel) # [cm/s by Thymio velocity unit]
# 
# # print(change)
# mean_speed = sum(speed)/len(speed)
# print(mean_speed)

In [12]:
# Rotation odometry

start = time.time()
th.set_var("motor.left.target", 100)
th.set_var("motor.right.target", 2**16 - 100)
time.sleep(1)

while (stop - start) < 9.32:
    stop = time.time()

th.set_var("motor.left.target", 0)
th.set_var("motor.right.target", 0)
print(stop - start)

NameError: name 'stop' is not defined

## Feel tor hinge

Markov localisation (see slide 21 of the Uncertainties lecture)

In [ ]:
def 